# Anexa 1

## Preprocesare

In [ ]:
pip install python_speech_features

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Importarea biblioteciilor

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import pywt
import scipy
from scipy.io import loadmat
from scipy.io import wavfile
from scipy.io.wavfile import write 
from scipy.signal import resample as scipy_resample
import python_speech_features

#### Constante

In [ ]:
wavelet_type = 'db10'

#### Functii utile

In [ ]:
def read_wav_scipy_wavfile(audio, dtype='float32'):
    """Loading audio and sample rate from a wav file, using wavfile.read. 


    Parameters
    ----------
    audio               : float32
                          multichannel/mono audio

    dtype               : deafult to float32                      
                          dtype for the resulted wav  

    Returns
    -------
    loaded_audio, sr    : tuple[float32, int]
                          audio with shape [channels, samples] and samplerate
    
    Observation! If the audio was written with a function, use the same function for load.
    """
    sr, loaded_audio = wavfile.read(audio)
    loaded_audio = loaded_audio.astype(dtype)
    return loaded_audio.T, sr

def resample_scipy(wav, old_sr, new_sr):
    """Load and resample audio using scipy.io wavfile and scipy.signal resample

    Parameters
    ----------
    wav                         : float32
                                multichannel/mono audio    
        
    old_sr                      : int
                                original samplerate                            
        
    new_sr                      : int                      
                                desired samplerate  

    Returns
    -------
    resampled_wav               : float32
                                 resampled audio with shape [channels, samples]
    
    Observation! If the audio was written with a function, use the same function for load.
    """
    new_no_samples = round(wav.shape[0] * new_sr / old_sr)
    resampled_wav = scipy_resample(wav.T, new_no_samples).T
    return resampled_wav


def normalize(audio):
     """Normalizes the audio, meaning the resulted samples range between -1 and 1.

     Parameters
     ----------
     audio               : float32
                           multichannel/stereo audio

     Returns
     -------
     normalized_audio    : float32
                           normalized multichannel/stereo audio
     """
     normalized_audio = audio/np.max(np.abs(audio))
     return normalized_audio

#### Peprocesare
- Segmentare
- Filtrare wavlet
- Normalizare
- Reeșantionare

In [ ]:
def preprocessing(stage, sr):
    '''doc string here'''

    new_sr = 1000
    wavelet_type = 'db10'
    
    stage = normalize(stage)
    stage = resample_scipy(stage, sr, new_sr)

    #DWTcoeffs = pywt.wavedec(stage, wavelet_type,mode='symmetric', level=4, axis=-1)
    #DWTcoeffs[-1] = np.zeros_like(DWTcoeffs[-1])
    #DWTcoeffs[-2] = np.zeros_like(DWTcoeffs[-2])
    #DWTcoeffs[-3] = np.zeros_like(DWTcoeffs[-3])
    #DWTcoeffs[-4] = np.zeros_like(DWTcoeffs[-4])

    #stage = pywt.waverec(DWTcoeffs, wavelet_type, mode='symmetric', axis=-1)
    
    return stage

## Extragerea caracteristicilor

Time-domain features:
- standard deviation
- Intervale:
	- S1 intervals, 
	- S2 intervals, 
	- systolic intervals, 
	- diastolic intervals, 
	- ratio of systolic interval to RR interval of each heart beat, 
	- ratio of diastolic interval to RR interval of each heart beat, 
	- ratio of systolic to diastolic interval of each heart beat.
- amplitudine:
	- ratio of the mean absolute amplitude during systole to that during the S1 period in each heart beat,
	- ratio of the mean absolute amplitude during diastole to that during the S2 period in each heart beat
	- skewness of the amplitude during S1 period in each heart beat, 
	- skewness of the amplitude during S2 period in each heart beat,
	- skewness of the amplitude during systole period in each heart beat, 
	- skewness of the amplitude during diastole period in each heart beat, 
	- kurtosis of the amplitude during S1 period in each heart beat, 
	- kurtosis of the amplitude during S2 period in each heart beat, 
	- kurtosis of the amplitude during systole period in each heart beat, 
	- kurtosis of the amplitude during diastole period in each heart beat

Frequency-domain Features:
The power spectrum of each heart sound state (i.e. S1, systole, S2, and diastole) was estimated using a Hamming window and the discrete-time Fourier transform.
- The median power across nine frequency bands (i.e. 25-45, 45-65, 65-85, 85-105, 105-125, 125-150, 150-200, 200-300, 300-400 Hz) corresponding to the S1, S2, systole, and diastole states of each cardiac cycle was calculated
- The mean of the median power of the nine frequency bands for all cycles were used as frequency-domain features (i.e. 9 frequency bands  4 states = 36 features)
- Additionally, 13 mel-frequency cepstral coefficient (MFCC) [6] were extracted from each state and each cardiac cycle.
- The mean of MFCCs across different cardiac cycles from the same heart sound recording was used as MFCC features (i.e. 13 MFCCs  4 states = 52 features). (MFCC = mel-frequesncy cepstrum)

Functii extragere caracteristici

In [ ]:
from scipy.fft import fft

def get_std(stage):
    """Get the standard deviation of the audio signal.

    Parameters
    ----------
    stage               : float32
                            audio with shape [samples, ]

    Returns
    -------
    std                 : float32
                            standard deviation of the audio signal

    """
    std = np.std(stage)
    return std

def get_interval(stage):
    """Get the interval of the audio signal.

    Parameters
    ----------
    stage               : float32
                            audio with shape [samples, ]


    Returns
    -------
    interval            : int
                            interval of the audio signal

    """
    interval = len(stage)
    return interval


def get_ratio(stage1, stage2):
    """Get the ratio of the audio signal.

    Parameters
    ----------
    stage               : float32
                            audio with shape [samples, ]


    Returns
    -------
    ratio               : float32
                            ratio of stage1/stage2

    """
    ratio = len(stage1) / len(stage2)
    return ratio


#ratio of the mean absolute amplitude during ______ to that during the ______ period in each heart beat
def ratio_means(signal_a, signal_b):
  ratio_means = (np.mean(abs(signal_a[1])))/(np.mean(abs(signal_b[1])))
  return ratio_means

#skewness
def skewness(signal):
    skewness = scipy.stats.skew(signal, axis = -1)
    return skewness

#kurtosis
def kurtosis(signal):
    kurtosis= scipy.stats.kurtosis(signal, axis = -1)
    return kurtosis


def get_median_power_in_frequency_band(stage, sr, start_freq, stop_freq):
    """Get the median power in a frequency band of the audio signal.

    Parameters
    ----------
    stage               : float32
                            audio with shape [samples, ]

    sr                  : int
                            sample rate of the audio signal

    start_freq          : int
                            start frequency of the frequency band

    stop_freq           : int
                            stop frequency of the frequency band

    Returns
    -------
    median_power        : float32
                            median power in the frequency band

    """

    # Perform the Fourier Transform
    freq_samples = fft(stage)

    # Convert the start and stop frequencies to corresponding bins
    start_bin = int(start_freq * len(stage) / sr)
    stop_bin = int(stop_freq * len(stage) / sr)

    # Calculate the power spectrum in the frequency band
    power_spectrum = np.abs(freq_samples[start_bin:stop_bin])**2

    # Calculate the median power across the frequency band
    median_power = np.median(power_spectrum)

    return median_power

def get_mean_of_medians(stage, sr, intervals = [[25, 45], [45, 65], [65, 85], [85, 105], 
                                                [105, 125], [125, 150], [150, 200], [200, 300], [300, 400]]):
    """Get the mean of power medians of the audio signal.

    Parameters
    ----------
    stage               : float32
                            audio with shape [samples, ]

    sr                  : int
                            sample rate of the audio signal
    
    intervals           : int
                            frequency bands of the audio signal
                            defaut parameter

    Returns
    -------
    mean_of_medians     : float32
                            mean of medians of the audio signal

    """

    power1 = get_median_power_in_frequency_band(stage, sr, intervals[0][0], intervals[0][1])
    power2 = get_median_power_in_frequency_band(stage, sr, intervals[1][0], intervals[1][1])
    power3 = get_median_power_in_frequency_band(stage, sr, intervals[2][0], intervals[2][1])
    power4 = get_median_power_in_frequency_band(stage, sr, intervals[3][0], intervals[3][1])
    power5 = get_median_power_in_frequency_band(stage, sr, intervals[4][0], intervals[4][1])
    power6 = get_median_power_in_frequency_band(stage, sr, intervals[5][0], intervals[5][1])
    power7 = get_median_power_in_frequency_band(stage, sr, intervals[6][0], intervals[6][1])
    power8 = get_median_power_in_frequency_band(stage, sr, intervals[7][0], intervals[7][1])
    power9 = get_median_power_in_frequency_band(stage, sr, intervals[8][0], intervals[8][1])

    mean_of_medians = (power1 + power2 + power3 + power4 + power5 + power6 + power7 + power8 + power9) / 9

    return mean_of_medians

def get_mean_of_medians_2(power1, power2, power3, power4, power5, power6, power7, power8, power9):
    mean_of_medians = (power1 + power2 + power3 + power4 + power5 + power6 + power7 + power8 + power9) / 9
    return mean_of_medians

def mel_coefficients(audio , rate):
    '''
    extract 13 dim mfcc features from an audio
    '''
    # get mfccs
    mfccs = python_speech_features.mfcc(audio, samplerate=rate, winlen=0.025, winstep=0.01,
                                      numcep=13, nfilt=26, nfft=512, lowfreq=0,
                                      highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=False)

    # compute mfcc means => 13 coef pentru fiecare
    mfcc_means = np.round(mfccs.mean(axis=0), 3)

    return mfcc_means


### preprocesare + extragere caracteristici

In [ ]:
columns = [    
               'signal',
               'label',
    
               'std_S1',
               'std_S2',
               'std_systole',
               'std_diastole',
    
               'interval_S1', 
               'interval_S2', 
               'interval_systole', 
               'interval_diastole',
    
               'systole/diastole_ratio', 
           
               'systole/S1_mean_amplit_ratio',
               'diastole/S2_mean_amplit_ratio',
               'skew_amplit_S1',
               'skew_amplit_S2',
               'skew_amplit_systole',
               'skew_amplit_diastole',
               'kurtosis_amplit_S1',
               'kurtosis_amplit_S2',
               'kurtosis_amplit_systole',
               'kurtosis_amplit_diastole',
               
               'meadian_pwr_25-45_S1',
               'meadian_pwr_45-65_S1',
               'meadian_pwr_65-85_S1',
               'meadian_pwr_85-105_S1',
               'meadian_pwr_105-125_S1',
               'meadian_pwr_125-150_S1',
               'meadian_pwr_150-200_S1',
               'meadian_pwr_200-300_S1',
               'meadian_pwr_300-400_S1',
           
               'meadian_pwr_25-45_S2',
               'meadian_pwr_45-65_S2',
               'meadian_pwr_65-85_S2',
               'meadian_pwr_85-105_S2',
               'meadian_pwr_105-125_S2',
               'meadian_pwr_125-150_S2',
               'meadian_pwr_150-200_S2',
               'meadian_pwr_200-300_S2',
               'meadian_pwr_300-400_S2',
           
               'meadian_pwr_25-45_systole',
               'meadian_pwr_45-65_systole',
               'meadian_pwr_65-85_systole',
               'meadian_pwr_85-105_systole',
               'meadian_pwr_105-125_systole',
               'meadian_pwr_125-150_systole',
               'meadian_pwr_150-200_systole',
               'meadian_pwr_200-300_systole',
               'meadian_pwr_300-400_systole',
               
               'meadian_pwr_25-45_diastole',
               'meadian_pwr_45-65_diastole',
               'meadian_pwr_65-85_diastole',
               'meadian_pwr_85-105_diastole',
               'meadian_pwr_105-125_diastole',
               'meadian_pwr_125-150_diastole',
               'meadian_pwr_150-200_diastole',
               'meadian_pwr_200-300_diastole',
               'meadian_pwr_300-400_diastole',
               
               'mean_medians_S1',
               'mean_medians_S2',
               'mean_medians_systole',
               'mean_medians_diastole',
           
               'mfcc_1_S1',
               'mfcc_2_S1',
               'mfcc_3_S1',
               'mfcc_4_S1',
               'mfcc_5_S1',
               'mfcc_6_S1',
               'mfcc_7_S1',
               'mfcc_8_S1',
               'mfcc_9_S1',
               'mfcc_10_S1',
               'mfcc_11_S1',
               'mfcc_12_S1',
               'mfcc_13_S1',
           
               'mfcc_1_S2',
               'mfcc_2_S2',
               'mfcc_3_S2',
               'mfcc_4_S2',
               'mfcc_5_S2',
               'mfcc_6_S2',
               'mfcc_7_S2',
               'mfcc_8_S2',
               'mfcc_9_S2',
               'mfcc_10_S2',
               'mfcc_11_S2',
               'mfcc_12_S2',
               'mfcc_13_S2',
           
               'mfcc_1_systole',
               'mfcc_2_systole',
               'mfcc_3_systole',
               'mfcc_4_systole',
               'mfcc_5_systole',
               'mfcc_6_systole',
               'mfcc_7_systole',
               'mfcc_8_systole',
               'mfcc_9_systole',
               'mfcc_10_systole',
               'mfcc_11_systole',
               'mfcc_12_systole',
               'mfcc_13_systole',
           
               'mfcc_1_diastole',
               'mfcc_2_diastole',
               'mfcc_3_diastole',
               'mfcc_4_diastole',
               'mfcc_5_diastole',
               'mfcc_6_diastole',
               'mfcc_7_diastole',
               'mfcc_8_diastole',
               'mfcc_9_diastole',
               'mfcc_10_diastole',
               'mfcc_11_diastole',
               'mfcc_12_diastole',
               'mfcc_13_diastole',
              ]

In [ ]:
def get_all_features(s1, s2, systole, diastole, sr=1000):
    """Extract the time domain, amplitude and frequency domain features from 
    the four stages of heart sound

    Parameters
    ----------
    s1                  : float32
                            audio with shape [samples, ]
    
    s2                  : float32
                            audio with shape [samples, ]
                            
    systole             : float32
                            audio with shape [samples, ]
                            
    diastole            : float32
                            audio with shape [samples, ]

    sr                  : int
                            sample rate of the audio signal

    Returns
    -------
    features            : dict of {str : float32}
                            dictionary of extracted features

    """
    features = {}
    # time domain
    features['std_S1'] = get_std(s1)
    features['std_S2'] = get_std(s2)
    features['std_systole'] = get_std(systole)
    features['std_diastole'] = get_std(diastole)
    
    features['interval_S1'] = get_interval(s1)
    features['interval_S2'] = get_interval(s2)
    features['interval_systole'] = get_interval(systole)
    features['interval_diastole'] = get_interval(diastole)
    
    features['systole/diastole_ratio'] = get_ratio(systole, diastole)

    #amplitude
    features['systole/S1_mean_amplit_ratio'] = ratio_means(systole, s1)
    features['diastole/S2_mean_amplit_ratio'] = ratio_means(diastole, s2)
    features['skew_amplit_S1'] = skewness(s1)
    features['skew_amplit_S2'] = skewness(s2)
    features['skew_amplit_systole'] = skewness(systole)
    features['skew_amplit_diastole'] = skewness(diastole)
    features['kurtosis_amplit_S1'] = kurtosis(s1)
    features['kurtosis_amplit_S2'] = kurtosis(s2)
    features['kurtosis_amplit_systole'] = kurtosis(systole)
    features['kurtosis_amplit_diastole'] = kurtosis(diastole)

    #frequency domain
    features['meadian_pwr_25-45_S1'] = get_median_power_in_frequency_band(s1, sr, 25, 45)
    features['meadian_pwr_45-65_S1'] = get_median_power_in_frequency_band(s1, sr, 45, 65)
    features['meadian_pwr_65-85_S1'] = get_median_power_in_frequency_band(s1, sr, 65, 85)
    features['meadian_pwr_85-105_S1'] = get_median_power_in_frequency_band(s1, sr, 85, 105)
    features['meadian_pwr_105-125_S1'] = get_median_power_in_frequency_band(s1, sr, 105, 125)
    features['meadian_pwr_125-150_S1'] = get_median_power_in_frequency_band(s1, sr, 125, 150)
    features['meadian_pwr_150-200_S1'] = get_median_power_in_frequency_band(s1, sr, 150, 200)
    features['meadian_pwr_200-300_S1'] = get_median_power_in_frequency_band(s1, sr, 200, 300)
    features['meadian_pwr_300-400_S1'] = get_median_power_in_frequency_band(s1, sr, 300, 400)

    features['meadian_pwr_25-45_S2'] = get_median_power_in_frequency_band(s2, sr, 25, 45)
    features['meadian_pwr_45-65_S2'] = get_median_power_in_frequency_band(s2, sr, 45, 65)
    features['meadian_pwr_65-85_S2'] = get_median_power_in_frequency_band(s2, sr, 65, 85)
    features['meadian_pwr_85-105_S2'] = get_median_power_in_frequency_band(s2, sr, 85, 105)
    features['meadian_pwr_105-125_S2'] = get_median_power_in_frequency_band(s2, sr, 105, 125)
    features['meadian_pwr_125-150_S2'] = get_median_power_in_frequency_band(s2, sr, 125, 150)
    features['meadian_pwr_150-200_S2'] = get_median_power_in_frequency_band(s2, sr, 150, 200)
    features['meadian_pwr_200-300_S2'] = get_median_power_in_frequency_band(s2, sr, 200, 300)
    features['meadian_pwr_300-400_S2'] = get_median_power_in_frequency_band(s2, sr, 300, 400)

    features['meadian_pwr_25-45_systole'] = get_median_power_in_frequency_band(systole, sr, 25, 45)
    features['meadian_pwr_45-65_systole'] = get_median_power_in_frequency_band(systole, sr, 45, 65)
    features['meadian_pwr_65-85_systole'] = get_median_power_in_frequency_band(systole, sr, 65, 85)
    features['meadian_pwr_85-105_systole'] = get_median_power_in_frequency_band(systole, sr, 85, 105)
    features['meadian_pwr_105-125_systole'] = get_median_power_in_frequency_band(systole, sr, 105, 125)
    features['meadian_pwr_125-150_systole'] = get_median_power_in_frequency_band(systole, sr, 125, 150)
    features['meadian_pwr_150-200_systole'] = get_median_power_in_frequency_band(systole, sr, 150, 200)
    features['meadian_pwr_200-300_systole'] = get_median_power_in_frequency_band(systole, sr, 200, 300)
    features['meadian_pwr_300-400_systole'] = get_median_power_in_frequency_band(systole, sr, 300, 400)

    features['meadian_pwr_25-45_diastole'] = get_median_power_in_frequency_band(diastole, sr, 25, 45)
    features['meadian_pwr_45-65_diastole'] = get_median_power_in_frequency_band(diastole, sr, 45, 65)
    features['meadian_pwr_65-85_diastole'] = get_median_power_in_frequency_band(diastole, sr, 65, 85)
    features['meadian_pwr_85-105_diastole'] = get_median_power_in_frequency_band(diastole, sr, 85, 105)
    features['meadian_pwr_105-125_diastole'] = get_median_power_in_frequency_band(diastole, sr, 105, 125)
    features['meadian_pwr_125-150_diastole'] = get_median_power_in_frequency_band(diastole, sr, 125, 150)
    features['meadian_pwr_150-200_diastole'] = get_median_power_in_frequency_band(diastole, sr, 150, 200)
    features['meadian_pwr_200-300_diastole'] = get_median_power_in_frequency_band(diastole, sr, 200, 300)
    features['meadian_pwr_300-400_diastole'] = get_median_power_in_frequency_band(diastole, sr, 300, 400)

    features['mean_medians_S1'] = get_mean_of_medians_2(
                                                        features['meadian_pwr_25-45_S1'],
                                                        features['meadian_pwr_45-65_S1'],
                                                        features['meadian_pwr_65-85_S1'],
                                                        features['meadian_pwr_85-105_S1'],
                                                        features['meadian_pwr_105-125_S1'],
                                                        features['meadian_pwr_125-150_S1'],
                                                        features['meadian_pwr_150-200_S1'],
                                                        features['meadian_pwr_200-300_S1'],
                                                        features['meadian_pwr_300-400_S1']
                                                    )
    features['mean_medians_S2'] = get_mean_of_medians_2(
                                                        features['meadian_pwr_25-45_S2'],
                                                        features['meadian_pwr_45-65_S2'],
                                                        features['meadian_pwr_65-85_S2'],
                                                        features['meadian_pwr_85-105_S2'],
                                                        features['meadian_pwr_105-125_S2'],
                                                        features['meadian_pwr_125-150_S2'],
                                                        features['meadian_pwr_150-200_S2'],
                                                        features['meadian_pwr_200-300_S2'],
                                                        features['meadian_pwr_300-400_S2']
                                                    )
    features['mean_medians_systole'] = get_mean_of_medians_2(
                                                        features['meadian_pwr_25-45_systole'],
                                                        features['meadian_pwr_45-65_systole'],
                                                        features['meadian_pwr_65-85_systole'],
                                                        features['meadian_pwr_85-105_systole'],
                                                        features['meadian_pwr_105-125_systole'],
                                                        features['meadian_pwr_125-150_systole'],
                                                        features['meadian_pwr_150-200_systole'],
                                                        features['meadian_pwr_200-300_systole'],
                                                        features['meadian_pwr_300-400_systole']
                                                    )
    features['mean_medians_diastole'] = get_mean_of_medians_2(
                                                        features['meadian_pwr_25-45_diastole'],
                                                        features['meadian_pwr_45-65_diastole'],
                                                        features['meadian_pwr_65-85_diastole'],
                                                        features['meadian_pwr_85-105_diastole'],
                                                        features['meadian_pwr_105-125_diastole'],
                                                        features['meadian_pwr_125-150_diastole'],
                                                        features['meadian_pwr_150-200_diastole'],
                                                        features['meadian_pwr_200-300_diastole'],
                                                        features['meadian_pwr_300-400_diastole']
                                                    )    
    (features['mfcc_1_S1'], features['mfcc_2_S1'], features['mfcc_3_S1'], features['mfcc_4_S1'], 
     features['mfcc_5_S1'], features['mfcc_6_S1'], features['mfcc_7_S1'], features['mfcc_8_S1'], 
     features['mfcc_9_S1'], features['mfcc_10_S1'], features['mfcc_11_S1'], features['mfcc_12_S1'], 
     features['mfcc_13_S1']) = mel_coefficients(s1, sr)

    (features['mfcc_1_S2'], features['mfcc_2_S2'], features['mfcc_3_S2'], features['mfcc_4_S2'], 
    features['mfcc_5_S2'], features['mfcc_6_S2'], features['mfcc_7_S2'], features['mfcc_8_S2'], 
    features['mfcc_9_S2'], features['mfcc_10_S2'], features['mfcc_11_S2'], features['mfcc_12_S2'], 
    features['mfcc_13_S2']) = mel_coefficients(s2, sr)

    (features['mfcc_1_systole'], features['mfcc_2_systole'], features['mfcc_3_systole'], features['mfcc_4_systole'], 
    features['mfcc_5_systole'], features['mfcc_6_systole'], features['mfcc_7_systole'], features['mfcc_8_systole'], 
    features['mfcc_9_systole'], features['mfcc_10_systole'], features['mfcc_11_systole'], features['mfcc_12_systole'], 
    features['mfcc_13_systole']) = mel_coefficients(systole, sr)

    (features['mfcc_1_diastole'], features['mfcc_2_diastole'], features['mfcc_3_diastole'], features['mfcc_4_diastole'], 
    features['mfcc_5_diastole'], features['mfcc_6_diastole'], features['mfcc_7_diastole'], features['mfcc_8_diastole'], 
    features['mfcc_9_diastole'], features['mfcc_10_diastole'], features['mfcc_11_diastole'], features['mfcc_12_diastole'], 
    features['mfcc_13_diastole']) = mel_coefficients(diastole, sr)
     
    return features

In [ ]:
# test pe o bataie
#diastole, _ = read_wav_scipy_wavfile(r'C:\Users\crist\Desktop\facultate\MAPSB\PCG_classification\one_beat\diastole_a0001_0.wav')
#s1, _ = read_wav_scipy_wavfile(r'C:\Users\crist\Desktop\facultate\MAPSB\PCG_classification\one_beat\s1_a0001_1.wav')
#systole, _ = read_wav_scipy_wavfile(r'C:\Users\crist\Desktop\facultate\MAPSB\PCG_classification\one_beat\systole_a0001_2.wav')
#s2, _ = read_wav_scipy_wavfile(r'C:\Users\crist\Desktop\facultate\MAPSB\PCG_classification\one_beat\s2_a0001_3.wav')

#features = get_all_features(s1, s2, systole, diastole, sr=1000)

#for (key, value) in features.items():
#    print(key, ':', value)

In [ ]:
#import pandas as pd
#df = pd.DataFrame(columns=columns)
#df = df.append(features, ignore_index=True)
#df.head()

In [ ]:
#training_sets = os.listdir("./data_wav")
#training_sets.sort()
#training_sets.pop()
training_sets = ["training-a", "training-b", "training-c", "training-d", "training-f"]

for directory in training_sets:
    print(directory)
    labels_path = f"./annotations/updated/{directory}/REFERENCE_withSQI.csv"
    labels_df = pd.read_csv(labels_path, header=None, names=["filename", "label", "quality"])
    df = pd.DataFrame(columns=columns)
    for filename in tqdm(os.listdir(f"./data_wav/{directory}")):    
            #print(filename)
            file_path = os.path.join(os.path.curdir, "data", directory, filename)
            annotations_path = os.path.join("annotations\\hand_corrected", 
                                    directory+"_StateAns", 
                                    filename.replace(".wav", "_StateAns.mat"))
            if (os.path.exists(annotations_path) == False):
                continue
            sr, signal = wavfile.read(file_path)
            annotations =  scipy.io.loadmat(annotations_path)['state_ans']
            i = 0
            while i < annotations.shape[0]-4:
                if ((annotations[i][1][0][0][0] == 'S1') and (annotations[i+1][1][0][0][0] == 'systole') and
                    (annotations[i+2][1][0][0][0] == 'S2') and (annotations[i+3][1][0][0][0] == 'diastole')):
                    #segmantare
                    s1_raw = signal[annotations[i][0][0][0]:annotations[i+1][0][0][0]]
                    systole_raw = signal[annotations[i+1][0][0][0]:annotations[i+2][0][0][0]]
                    s2_raw = signal[annotations[i+2][0][0][0]:annotations[i+3][0][0][0]]
                    diastole_raw = signal[annotations[i+3][0][0][0]:annotations[i+4][0][0][0]]
                    
                    #preprocesare
                    s1 = preprocessing(s1_raw, sr)
                    systole = preprocessing(systole_raw, sr)
                    s2 = preprocessing(s2_raw, sr)
                    diastole = preprocessing(diastole_raw, sr)
                    
                    #extargere caracteristici
                    features = get_all_features(s1=s1, s2=s2, systole=systole, diastole=diastole, sr=1000)
                    features['signal'] = filename
                    features['label'] = labels_df[labels_df["filename"] == filename.split('.')[0]]["label"].values[0]
                    
                    df = df.append(features, ignore_index=True)
                    # break;
                    i+=4
                else:
                    i+=1
            df.to_csv(f'./features/{directory}.csv', index=False)
            # break; 
        

  0%|                                                                                          | 0/409 [00:00<?, ?it/s]

training-a


  0%|▏                                                                                 | 1/490 [00:00<01:14,  6.54it/s]

training-b


  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

...

  0%|                                                                                           | 0/55 [00:00<?, ?it/s]

training-d


  0%|                                                                                          | 0/114 [00:00<?, ?it/s]

training-f


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [01:29<00:00,  1.27it/s]


In [ ]:
training_sets = ["training-e"]

for directory in training_sets:
    print(directory)
    labels_path = f"./annotations/updated/{directory}/REFERENCE_withSQI.csv"
    labels_df = pd.read_csv(labels_path, header=None, names=["filename", "label", "quality"])
    df = pd.DataFrame(columns=columns)
    for filename in tqdm(os.listdir(f"./data_wav/{directory}")):    
            #print(filename)
            file_path = os.path.join(os.path.curdir, "data", directory, filename)
            annotations_path = os.path.join("annotations\\hand_corrected", 
                                    directory+"_StateAns", 
                                    filename.replace(".wav", "_StateAns.mat"))
            if (os.path.exists(annotations_path) == False):
                continue
            sr, signal = wavfile.read(file_path)
            annotations =  scipy.io.loadmat(annotations_path)['state_ans']
            i = 0
            while i < annotations.shape[0]-4:
                if ((annotations[i][1][0][0][0] == 'S1') and (annotations[i+1][1][0][0][0] == 'systole') and
                    (annotations[i+2][1][0][0][0] == 'S2') and (annotations[i+3][1][0][0][0] == 'diastole')):
                    #segmantare
                    s1_raw = signal[annotations[i][0][0][0]:annotations[i+1][0][0][0]]
                    systole_raw = signal[annotations[i+1][0][0][0]:annotations[i+2][0][0][0]]
                    s2_raw = signal[annotations[i+2][0][0][0]:annotations[i+3][0][0][0]]
                    diastole_raw = signal[annotations[i+3][0][0][0]:annotations[i+4][0][0][0]]
                    
                    #preprocesare
                    s1 = preprocessing(s1_raw, sr)
                    systole = preprocessing(systole_raw, sr)
                    s2 = preprocessing(s2_raw, sr)
                    diastole = preprocessing(diastole_raw, sr)
                    
                    #extargere caracteristici
                    features = get_all_features(s1=s1, s2=s2, systole=systole, diastole=diastole, sr=1000)
                    features['signal'] = filename
                    features['label'] = labels_df[labels_df["filename"] == filename.split('.')[0]]["label"].values[0]
                    
                    df = df.append(features, ignore_index=True)
                    # break;
                    i+=4
                else:
                    i+=1
            df.to_csv(f'./features/{directory}.csv', index=False)
            # break; 

  0%|                                                                                         | 0/2141 [00:00<?, ?it/s]

training-e


 11%|████████▍                                                                      | 230/2141 [02:40<29:48,  1.07it/s]<ipython-input-45-fffd2f81d6d2>:63: RuntimeWarning: invalid value encountered in true_divide
  normalized_audio = audio/np.max(np.abs(audio))
 60%|█████████████████████████████████████████████▌                              | 1284/2141 [47:03<1:07:51,  4.75s/it]